In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Pomyślmy krok po kroku."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="mistral")

chain = prompt | model

chain.invoke({"question": "Czym jest LangChain?"})

'1. Co to oznacza "LangChain"?\n\n2. Jest to nazwa jakiejś większej jednostki lub platformy, ale nie wiem szczegółowo.\n\n3. Możesz dodać, że jest to może program, narzędzie czy serwis internetowy, lecz nie wiem dokładniej.\n\n4. Aby dowiedzieć się więcej mamy kilka opcji:\n- Sprawdzenie strony WWW, gdy jesteś pewien, że jest coś na ten temat na Internetu.\n- Zadanie pytania w internecie, aby uzyskać szczegółowe informacje o LangChainie.\n- Skontaktowanie się z firmą lub osobą która go stworzyła, jeśli jesteś pewien, że jest ktoś taki.\n- Przeglądanie literatury fachowej lub artykułów na ten temat.\n\n5. Wybierz jedną z powyższych opcji i dowiedz się więcej o LangChainie!'

In [3]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="mistral")

In [16]:
from langchain_qdrant import QdrantVectorStore, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="papers_collection",
    vectors_config=VectorParams(size=4096, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="papers_collection",
    embedding=embeddings,
    retrieval_mode=RetrievalMode.DENSE
)

In [17]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_chunks(pdf_path):
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

    return text_splitter.split_documents(documents)

In [18]:
def vectorize_pdf_content(pdf_path):
    chunks = load_chunks(pdf_path)
    _ = vector_store.add_documents(documents=chunks)

pdf_path = "Aspirin C.pdf"
vectorize_pdf_content(pdf_path)

In [19]:
from langchain_core.runnables import RunnablePassthrough

retriever = vector_store.as_retriever(search_kwargs={"k": 3})
retriever_runnable = {"question": RunnablePassthrough(), "context": retriever}

In [34]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import SystemMessage

prompt = hub.pull("rlm/rag-prompt")

prompt_instruction = SystemMessage(
    content = "Zawsze odpowiadaj w języku, w którym otrzymałeś pytanie. " +
    "Udzielaj zwięzłych odpowiedzi i upewnij się, że twoja odpowiedź jest mocno powiązana z dostarczonym kontekstem."
)
updated_messages = [prompt_instruction] + prompt.messages
prompt.messages = updated_messages

generator_runnable = prompt | model | StrOutputParser()


/Users/mwardynski/Documents/ds/_semestr_9/przetwarzanie_jezyka_naturalnego/labs/lab10/.venv/lib/python3.12/site-packages/langsmith/client.py:256: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [35]:
chain = retriever_runnable | generator_runnable

In [36]:
question = "dla kogo jest to lek?"

result = chain.invoke(question)
print(result)

1. Lek Aspirin C jest przeznaczony dla dorosłych.
2. Składnikami czynnymi leku Aspirin C są kwas acetylosalicylowy i witamina C, a pozostałe składniki to sodu cytrynian bezwodny, sodu wodorowęglan, kwas cytrynowy bezwodny oraz sodu węglan bezwodny.
3. Pacjentom ze zmniejszoną czynnością nerek i pacjentom kontrolujących zawartość sodu w diecie należy podążać za zaleceniami lekarza lub farmaceuty w stosowaniu leku Aspirin C.
